### Topic modelling

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time 
import gensim.corpora as corpora
from gensim.models import LdaMulticore
import os, re
sub_dir = '/Users/dominicbates/Documents/GitHub/app-review-classifier/'
os.chdir(sub_dir)

from app_review_classifier.text_processing import TextCleaner, Embedder

In [2]:
reviews_df = pd.read_parquet('/Users/dominicbates/Documents/GitHub/app-review-classifier/data/reviews_sample.parquet')


In [48]:
# Get cleaner object
cleaner = TextCleaner(config={'stop_words':True,'ngrams':True,'lemmatization':True})
cleaner.stop_words = cleaner.stop_words | {'economist', 'use', 'using', 'app','apps','read','reading', 'reads', 'article', 'articles'}

# Clean text
cleaned_reviews = cleaner.process_raw_text(reviews_df['review'], train_ngrams = True)

# Remove really short reviews
print('\nOriginal N. reviews:',len(cleaned_reviews))
cleaned_reviews = [review for review in cleaned_reviews if len(review)>=8]
print('After removing short reviews:',len(cleaned_reviews))


Cleaning up text and removing stopwords...
- Text cleaned in: 0.27 seconds


INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:PROGRESS: at sentence #10000, processed 131399 words and 95659 word types



Training ngrams...


INFO:gensim.models.phrases:collected 113089 token types (unigram + bigrams) from a corpus of 169373 words and 14986 sentences
INFO:gensim.models.phrases:merged Phrases<113089 vocab, min_count=1, threshold=200, max_vocab_size=40000000>
INFO:gensim.utils:Phrases lifecycle event {'msg': 'built Phrases<113089 vocab, min_count=1, threshold=200, max_vocab_size=40000000> in 0.23s', 'datetime': '2022-08-18T11:59:15.540100', 'gensim': '4.2.0', 'python': '3.7.4 (default, Sep  7 2019, 18:27:02) \n[Clang 10.0.1 (clang-1001.0.46.4)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
INFO:gensim.models.phrases:collecting all words and their counts
INFO:gensim.models.phrases:PROGRESS: at sentence #0, processed 0 words and 0 word types
INFO:gensim.models.phrases:PROGRESS: at sentence #10000, processed 128916 words and 96101 word types
INFO:gensim.models.phrases:collected 113750 token types (unigram + bigrams) from a corpus of 166182 words and 14986 sentences
INFO:gensim.models.phrase

- Training done in: 1.4 seconds

Loading spacy model...
- Model loaded in: 0.57 seconds

Lemmatizing...
- Lemmatizing done in: 37.56 seconds

Original N. reviews: 14986
After removing short reviews: 6801


In [4]:
# test = Embedder(method='word2vec')

In [16]:
# test.fit(cleaned_reviews)
# cleaner.stop_words

In [102]:
default_configs = {'lda':{'num_topics':8,
                          'passes':300, #2000, # Default 1 just tried 50
                          'iterations':100, # Default 50
                          'random_state':1,
                          'eval_every':10}}

class TopicModel:
    
    def __init__(self, 
                 config=None):
        
        # Set config params
        if config is None:
            self.config = default_configs['lda']
        else:
            self.config = config
            
        self.model = None
        self.num_topics = self.config['num_topics']
        self.training_passes = self.config['passes']
        self.training_iterations = self.config['iterations']
        self.training_random_state = self.config['random_state']
        self.training_eval_every = self.config['eval_every']

    
    def get_corpus(self, documents, train_dict=False):
        print('Getting corpus...')
        t1=time.time()
        if train_dict==True:
            print('- Training dictionary...')
            self.id2word = corpora.Dictionary(documents)
            
            if self.id2word is None:
                raise ValueError('No dictionary trained yet. Try ruynning get_corpus() with train=True')
            
        print('- Done in: {} seconds'.format(round((time.time() - t1), 2)))
        return [self.id2word.doc2bow(text) for text in cleaned_reviews]
                    

    def fit(self, documents):

        corpus = self.get_corpus(documents, train_dict=True)
        
        print('\nTraining model...')
        t1=time.time()
        self.model = LdaMulticore(corpus=corpus,
                                  id2word=self.id2word,
                                  num_topics=self.num_topics,
                                  passes = self.training_passes, # Default 1 just tried 50
                                  iterations = self.training_iterations, # Default 50
                                  random_state=self.training_random_state,
                                  eval_every=self.training_eval_every)        
        print('- Model trained in: {} seconds'.format(round((time.time() - t1), 2)))


    def apply(self, documents):
        corpus = self.get_corpus(documents, train_dict=False)


In [103]:
import logging
logging.basicConfig(level=logging.INFO)

test_model = TopicModel()
test_model.fit(cleaned_reviews)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary<0 unique tokens: []>


Getting corpus...
- Training dictionary...


INFO:gensim.corpora.dictionary:built Dictionary<8622 unique tokens: ['ability', 'actually', 'apple', 'apply', 'bottom']...> from 6801 documents (total 114124 corpus positions)
INFO:gensim.utils:Dictionary lifecycle event {'msg': "built Dictionary<8622 unique tokens: ['ability', 'actually', 'apple', 'apply', 'bottom']...> from 6801 documents (total 114124 corpus positions)", 'datetime': '2022-08-18T13:21:42.066987', 'gensim': '4.2.0', 'python': '3.7.4 (default, Sep  7 2019, 18:27:02) \n[Clang 10.0.1 (clang-1001.0.46.4)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}
INFO:gensim.models.ldamodel:using symmetric alpha at 0.125
INFO:gensim.models.ldamodel:using symmetric eta at 0.125
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 8 topics, 300 passes over the supplied corpus of 6801 documents, updating every 14000 documents, evaluating every ~6801 documents, iterating 100x with a convergenc

- Done in: 0.24 seconds

Training model...


INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/6801, outstanding queue size 1
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/6801, outstanding queue size 2
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/6801, outstanding queue size 3
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #3 = documents up to #6801/6801, outstanding queue size 4
INFO:gensim.models.ldamodel:topic #7 (0.125): 0.014*"time" + 0.013*"day" + 0.012*"great" + 0.009*"new" + 0.009*"news" + 0.009*"load" + 0.009*"very" + 0.009*"work" + 0.008*"easy" + 0.008*"download"
INFO:gensim.models.ldamodel:topic #1 (0.125): 0.014*"time" + 0.014*"great" + 0.012*"work" + 0.011*"news" + 0.011*"audio" + 0.011*"update" + 0.011*"issue" + 0.009*"subscription" + 0.009*"story" + 0.009*"download"
INFO:gensim.models.ldamodel:topic #0 (0.125): 0.019*"great" + 0.013*"news" + 0.012*"morning

...

INFO:gensim.models.ldamodel:topic #1 (0.125): 0.045*"subscription" + 0.023*"pay" + 0.016*"ad" + 0.014*"work" + 0.013*"content" + 0.013*"time" + 0.012*"access" + 0.011*"cancel" + 0.011*"try" + 0.010*"subscribe"
INFO:gensim.models.ldamodel:topic #7 (0.125): 0.020*"load" + 0.018*"time" + 0.016*"open" + 0.015*"screen" + 0.011*"page" + 0.011*"content" + 0.010*"version" + 0.010*"work" + 0.010*"only" + 0.009*"issue"
INFO:gensim.models.ldamodel:topic diff=0.000378, rho=0.058783
INFO:gensim.models.ldamodel:-7.273 per-word bound, 154.7 perplexity estimate based on a held-out corpus of 801 documents with 10277 words
INFO:gensim.models.ldamulticore:PROGRESS: pass 286, dispatched chunk #0 = documents up to #2000/6801, outstanding queue size 1
INFO:gensim.models.ldamulticore:PROGRESS: pass 286, dispatched chunk #1 = documents up to #4000/6801, outstanding queue size 2
INFO:gensim.models.ldamulticore:PROGRESS: pass 286, dispatched chunk #2 = documents up to #6000/6801, outstanding queue size 3
INFO:g

INFO:gensim.models.ldamodel:topic #4 (0.125): 0.014*"news" + 0.013*"always" + 0.012*"magazine" + 0.009*"truly" + 0.008*"year" + 0.006*"enjoy" + 0.006*"write" + 0.005*"smoothly" + 0.005*"agree" + 0.005*"report"
INFO:gensim.models.ldamodel:topic #3 (0.125): 0.047*"audio" + 0.025*"download" + 0.018*"listen" + 0.017*"version" + 0.015*"content" + 0.014*"play" + 0.013*"edition" + 0.012*"new" + 0.011*"work" + 0.010*"issue"
INFO:gensim.models.ldamodel:topic #0 (0.125): 0.016*"more" + 0.015*"love" + 0.011*"year" + 0.010*"issue" + 0.010*"great" + 0.007*"think" + 0.007*"magazine" + 0.007*"copy" + 0.006*"print" + 0.006*"good"
INFO:gensim.models.ldamodel:topic #1 (0.125): 0.045*"subscription" + 0.023*"pay" + 0.016*"ad" + 0.014*"work" + 0.013*"content" + 0.013*"time" + 0.012*"access" + 0.011*"cancel" + 0.011*"try" + 0.010*"subscribe"
INFO:gensim.models.ldamodel:topic diff=0.000376, rho=0.058282
INFO:gensim.models.ldamodel:-7.273 per-word bound, 154.7 perplexity estimate based on a held-out corpus of

INFO:gensim.models.ldamulticore:PROGRESS: pass 295, dispatched chunk #2 = documents up to #6000/6801, outstanding queue size 3
INFO:gensim.models.ldamulticore:PROGRESS: pass 295, dispatched chunk #3 = documents up to #6801/6801, outstanding queue size 4
INFO:gensim.models.ldamodel:topic #1 (0.125): 0.045*"subscription" + 0.023*"pay" + 0.016*"ad" + 0.014*"work" + 0.013*"content" + 0.013*"time" + 0.012*"access" + 0.011*"cancel" + 0.011*"try" + 0.010*"subscribe"
INFO:gensim.models.ldamodel:topic #0 (0.125): 0.016*"more" + 0.015*"love" + 0.011*"year" + 0.010*"issue" + 0.009*"great" + 0.007*"think" + 0.007*"magazine" + 0.007*"copy" + 0.006*"print" + 0.006*"hard"
INFO:gensim.models.ldamodel:topic #7 (0.125): 0.020*"load" + 0.018*"time" + 0.016*"open" + 0.015*"screen" + 0.011*"page" + 0.011*"content" + 0.010*"version" + 0.010*"work" + 0.010*"only" + 0.009*"issue"
INFO:gensim.models.ldamodel:topic #3 (0.125): 0.047*"audio" + 0.025*"download" + 0.018*"listen" + 0.017*"version" + 0.015*"content"

INFO:gensim.models.ldamodel:topic diff=0.000336, rho=0.057411
INFO:gensim.models.ldamodel:-7.273 per-word bound, 154.7 perplexity estimate based on a held-out corpus of 801 documents with 10277 words
INFO:gensim.utils:LdaMulticore lifecycle event {'msg': 'trained LdaMulticore<num_terms=8622, num_topics=8, decay=0.5, chunksize=2000> in 313.00s', 'datetime': '2022-08-18T13:26:55.188640', 'gensim': '4.2.0', 'python': '3.7.4 (default, Sep  7 2019, 18:27:02) \n[Clang 10.0.1 (clang-1001.0.46.4)]', 'platform': 'Darwin-19.6.0-x86_64-i386-64bit', 'event': 'created'}


- Model trained in: 313.01 seconds


In [104]:
# '''
# PASS 25
# INFO:gensim.models.ldamodel:topic #0 (0.167): 0.018*"content" + 0.016*"time" + 0.013*"work" + 0.013*"new" + 0.013*"update" + 0.012*"open" + 0.010*"fix" + 0.009*"load" + 0.009*"version" + 0.008*"issue"
# INFO:gensim.models.ldamodel:topic #1 (0.167): 0.036*"subscription" + 0.020*"pay" + 0.012*"content" + 0.011*"ad" + 0.010*"very" + 0.010*"cancel" + 0.009*"only" + 0.009*"more" + 0.008*"access" + 0.008*"good"
# INFO:gensim.models.ldamodel:topic #2 (0.167): 0.049*"audio" + 0.025*"download" + 0.018*"listen" + 0.015*"play" + 0.014*"edition" + 0.013*"version" + 0.012*"work" + 0.011*"new" + 0.010*"issue" + 0.009*"feature"
# INFO:gensim.models.ldamodel:topic #3 (0.167): 0.041*"news" + 0.033*"world" + 0.027*"morning" + 0.025*"day" + 0.023*"great" + 0.015*"daily" + 0.015*"event" + 0.014*"way" + 0.012*"quick" + 0.012*"good"
# INFO:gensim.models.ldamodel:topic #4 (0.167): 0.033*"news" + 0.030*"day" + 0.016*"story" + 0.016*"great" + 0.012*"more" + 0.010*"source" + 0.009*"way" + 0.009*"write" + 0.009*"only" + 0.009*"easy"


# PASS 50
# INFO:gensim.models.ldamodel:topic #0 (0.167): 0.019*"content" + 0.017*"time" + 0.014*"update" + 0.014*"work" + 0.013*"open" + 0.013*"new" + 0.011*"fix" + 0.011*"load" + 0.009*"issue" + 0.008*"version"
# INFO:gensim.models.ldamodel:topic #1 (0.167): 0.042*"subscription" + 0.023*"pay" + 0.013*"ad" + 0.012*"content" + 0.011*"cancel" + 0.009*"only" + 0.009*"very" + 0.009*"access" + 0.009*"subscribe" + 0.008*"more"
# INFO:gensim.models.ldamodel:topic #2 (0.167): 0.050*"audio" + 0.026*"download" + 0.019*"listen" + 0.016*"play" + 0.015*"edition" + 0.015*"version" + 0.012*"work" + 0.012*"new" + 0.010*"issue" + 0.010*"feature"
# INFO:gensim.models.ldamodel:topic #4 (0.167): 0.030*"news" + 0.025*"day" + 0.017*"story" + 0.013*"more" + 0.012*"great" + 0.011*"source" + 0.011*"write" + 0.010*"easy" + 0.009*"love" + 0.008*"only"
# INFO:gensim.models.ldamodel:topic #5 (0.167): 0.021*"good" + 0.021*"news" + 0.012*"very" + 0.011*"content" + 0.010*"great" + 0.010*"time" + 0.009*"more" + 0.008*"issue" + 0.008*"version" + 0.007*"magazine"


# PASS 100
# INFO:gensim.models.ldamodel:topic #0 (0.167): 0.019*"content" + 0.017*"time" + 0.015*"update" + 0.014*"work" + 0.013*"open" + 0.013*"new" + 0.011*"fix" + 0.011*"load" + 0.010*"issue" + 0.009*"great"
# INFO:gensim.models.ldamodel:topic #2 (0.167): 0.050*"audio" + 0.025*"download" + 0.019*"listen" + 0.016*"play" + 0.015*"edition" + 0.015*"version" + 0.012*"new" + 0.012*"work" + 0.010*"feature" + 0.010*"issue"
# INFO:gensim.models.ldamodel:topic #3 (0.167): 0.047*"news" + 0.031*"world" + 0.030*"day" + 0.029*"morning" + 0.027*"great" + 0.017*"daily" + 0.016*"way" + 0.014*"good" + 0.013*"quick" + 0.012*"event"
# INFO:gensim.models.ldamodel:topic #4 (0.167): 0.028*"news" + 0.018*"day" + 0.017*"story" + 0.013*"more" + 0.012*"source" + 0.012*"write" + 0.010*"easy" + 0.009*"great" + 0.009*"love" + 0.007*"provide"
# INFO:gensim.models.ldamodel:topic #5 (0.167): 0.021*"good" + 0.020*"news" + 0.011*"content" + 0.010*"very" + 0.010*"more" + 0.009*"time" + 0.008*"great" + 0.008*"version" + 0.008*"issue" + 0.008*"magazine"

# PASS 150 

# INFO:gensim.models.ldamodel:topic #0 (0.167): 0.019*"content" + 0.017*"time" + 0.015*"update" + 0.014*"work" + 0.013*"open" + 0.012*"new" + 0.011*"fix" + 0.011*"load" + 0.010*"issue" + 0.009*"great"
# INFO:gensim.models.ldamodel:topic #1 (0.167): 0.048*"subscription" + 0.025*"pay" + 0.015*"ad" + 0.012*"cancel" + 0.011*"content" + 0.011*"subscribe" + 0.010*"access" + 0.009*"only" + 0.009*"digital" + 0.009*"very"
# INFO:gensim.models.ldamodel:topic #2 (0.167): 0.050*"audio" + 0.025*"download" + 0.019*"listen" + 0.016*"play" + 0.016*"version" + 0.015*"edition" + 0.013*"new" + 0.012*"work" + 0.011*"feature" + 0.010*"issue"
# INFO:gensim.models.ldamodel:topic #4 (0.167): 0.027*"news" + 0.016*"story" + 0.015*"day" + 0.013*"more" + 0.012*"write" + 0.012*"source" + 0.009*"easy" + 0.008*"love" + 0.007*"provide" + 0.007*"great"
# INFO:gensim.models.ldamodel:topic #5 (0.167): 0.022*"good" + 0.020*"news" + 0.010*"content" + 0.010*"more" + 0.009*"very" + 0.008*"time" + 0.008*"magazine" + 0.007*"version" + 0.007*"issue" + 0.007*"great"


# '''        # default_configs = {'lda':{num_topics=10,
# #                           passes = 2000, # Default 1 just tried 50
# #                           iterations = 500, # Default 50
# #                           random_state=1234}}


# # print('Training model...')
# # t1=time.time()
# # lda_model = LdaMulticore(corpus=corpus,
# #                          id2word=id2word,
# #                          num_topics=10,
# #                          passes = 2000, # Default 1 just tried 50
# #                          iterations = 500, # Default 50
# #                          random_state=1234,
# #                          eval_every=10)
# # print('Model trained in: {} seconds'.format(round((time.time() - t1), 2)))



In [105]:
test = test_model.model.print_topics()
for n in range(len(test)):
    print('Topic',test[n][0],'=',[w for w in test[n][1].split('"') if "*" not in w][:-1])

INFO:gensim.models.ldamodel:topic #0 (0.125): 0.016*"more" + 0.015*"love" + 0.011*"year" + 0.010*"issue" + 0.009*"great" + 0.007*"think" + 0.007*"magazine" + 0.007*"copy" + 0.006*"print" + 0.006*"hard"
INFO:gensim.models.ldamodel:topic #1 (0.125): 0.045*"subscription" + 0.023*"pay" + 0.016*"ad" + 0.014*"work" + 0.013*"content" + 0.013*"time" + 0.012*"access" + 0.011*"cancel" + 0.011*"try" + 0.010*"subscribe"
INFO:gensim.models.ldamodel:topic #2 (0.125): 0.051*"news" + 0.027*"day" + 0.024*"world" + 0.024*"great" + 0.022*"morning" + 0.016*"good" + 0.014*"daily" + 0.014*"way" + 0.011*"story" + 0.010*"more"
INFO:gensim.models.ldamodel:topic #3 (0.125): 0.047*"audio" + 0.025*"download" + 0.018*"listen" + 0.017*"version" + 0.015*"content" + 0.014*"play" + 0.013*"edition" + 0.012*"new" + 0.011*"work" + 0.010*"issue"
INFO:gensim.models.ldamodel:topic #4 (0.125): 0.014*"news" + 0.013*"always" + 0.012*"magazine" + 0.009*"truly" + 0.008*"year" + 0.006*"enjoy" + 0.006*"write" + 0.005*"smoothly" + 

Topic 0 = ['more', 'love', 'year', 'issue', 'great', 'think', 'magazine', 'copy', 'print', 'hard']
Topic 1 = ['subscription', 'pay', 'ad', 'work', 'content', 'time', 'access', 'cancel', 'try', 'subscribe']
Topic 2 = ['news', 'day', 'world', 'great', 'morning', 'good', 'daily', 'way', 'story', 'more']
Topic 3 = ['audio', 'download', 'listen', 'version', 'content', 'play', 'edition', 'new', 'work', 'issue']
Topic 4 = ['news', 'always', 'magazine', 'truly', 'year', 'enjoy', 'write', 'smoothly', 'agree', 'report']
Topic 5 = ['sometimes', 'crn', 'perspective', 'notification', 'usual', 'find', 'user', 'activate', 'page', 'run']
Topic 6 = ['more', 'new', 'content', 'update', 'version', 'good', 'feature', 'phone', 'really', 'need']
Topic 7 = ['load', 'time', 'open', 'screen', 'page', 'content', 'version', 'work', 'only', 'issue']


### Test run 1

In [71]:
'''
default_configs = {'lda':{'num_topics':6,
                          'passes':200, #2000, # Default 1 just tried 50
                          'iterations':100, # Default 50
                          'random_state':1234,
                          'eval_every':10}}
                          
                          
Topic 0 = ['content', 'time', 'update', 'work', 'open', 'new', 'load', 'fix', 'issue', 'great']
Topic 1 = ['subscription', 'pay', 'ad', 'cancel', 'content', 'subscribe', 'access', 'only', 'digital', 'very']
Topic 2 = ['audio', 'download', 'listen', 'version', 'play', 'edition', 'new', 'work', 'feature', 'issue']
Topic 3 = ['news', 'day', 'world', 'morning', 'great', 'daily', 'way', 'good', 'quick', 'event']
Topic 4 = ['news', 'story', 'day', 'more', 'write', 'source', 'easy', 'love', 'provide', 'great']
Topic 5 = ['good', 'news', 'more', 'content', 'very', 'magazine', 'time', 'version', 'issue', 'year']


Topic 0 = Bugs/Issues?
Topic 1 = Payment/subscription?
Topic 2 = Audio edition
Topic 3 = Espresso / world in brief
Topic 4 = Journalism
Topic 5 = Weekly edition?

''';

SyntaxError: invalid syntax (<ipython-input-71-01dfcb5b7f94>, line 1)

### Test run 2/3 (same result)

In [ ]:
'''
default_configs = {'lda':{'num_topics':6,
                          'passes':200, #2000, # Default 1 just tried 50
                          'iterations':100, # Default 50
                          'random_state':1, # Also tried 123
                          'eval_every':10}}
                          
Topic 0 = ['more', 'great', 'love', 'issue', 'year', 'very', 'find', 'only', 'always', 'thing']
Topic 1 = ['subscription', 'pay', 'time', 'work', 'content', 'ad', 'try', 'update', 'open', 'only']
Topic 2 = ['news', 'day', 'world', 'great', 'morning', 'good', 'daily', 'way', 'more', 'story']
Topic 3 = ['audio', 'download', 'version', 'content', 'new', 'listen', 'edition', 'play', 'old', 'work']
Topic 4 = ['magazine', 'always', 'year', 'enjoy', 'smoothly', 'well', 'challenge', 'support', 'write', 'true']
Topic 5 = ['truly', 'present', 'new', 'user', 'little', 'perspective', 'sense', 'year', 'compliment', 'experience']

Topic 0 = ?
Topic 1 = Payment/subscription?
Topic 2 = Espresso / world in brief
Topic 3 = Audio edition
Topic 4 = Weekly edition?
Topic 5 = Quality Journalism

'''

### Test run 1 with 10 topics

In [ ]:
'''
default_configs = {'lda':{'num_topics':6,
                          'passes':200, #2000, # Default 1 just tried 50
                          'iterations':100, # Default 50
                          'random_state':1, # Also tried 123
                          'eval_every':10}}
                          
Topic 0 = ['love', 'great', 'year', 'issue', 'more', 'thing', 'copy', 'only', 'market', 'think'] - Weekly Edition?
Topic 1 = ['update', 'time', 'work', 'issue', 'log', 'open', 'try', 'fix', 'login', 'subscription'] - Bugs (login)
Topic 2 = ['news', 'world', 'good', 'day', 'daily', 'event', 'source', 'write', 'provide', 'global'] - News (espresso?)
Topic 3 = ['audio', 'download', 'listen', 'version', 'edition', 'play', 'new', 'content', 'work', 'feature'] - Audio
Topic 4 = ['news', 'writing', 'thank', 'enjoy', 'always', 'reporting', 'year', 'professional', 'report', 'magazine'] - Quality Journalism
Topic 5 = ['source', 'news', 'thank', 'fact', 'trust', 'quality', 'value', 'provide', 'analysis', 'objective'] - Quality Journalism
Topic 6 = ['new', 'more', 'content', 'year', 'hour', 'feature', 'take', 'need', 'news', 'word'] - ?
Topic 7 = ['load', 'time', 'screen', 'open', 'content', 'slow', 'scroll', 'text', 'very', 'page'] - Bugs (slow?)
Topic 8 = ['subscription', 'pay', 'ad', 'content', 'access', 'cancel', 'version', 'new', 'subscribe', 'more'] - Subscription / pricing / too many ads
Topic 9 = ['news', 'great', 'morning', 'day', 'way', 'world', 'easy', 'daily', 'quick', 'more'] - Espresso / word in brief


Topic 0 = ? / Weekly Edition
Topic 1 = Bugs / login
Topic 2 = News/espresso?
Topic 3 = Audio
Topic 4 = Quality Journalism
Topic 5 = Quality Journalism
Topic 6 = ?
Topic 7 = Bugs / slow
Topic 8 = Subscription/pricing
Topic 9 = Espresso / word in brief



'''

### Test run 1 with 15 topics

In [ ]:
'''

default_configs = {'lda':{'num_topics':15,
                          'passes':500, #2000, # Default 1 just tried 50
                          'iterations':100, # Default 50
                          'random_state':1,
                          'eval_every':10}}


Topic 0 = ['love', 'great', 'more', 'issue', 'year', 'work', 'magazine', 'good', 'think', 'very']
Topic 1 = ['subscription', 'work', 'time', 'try', 'log', 'update', 'access', 'fix', 'download', 'login']
Topic 2 = ['news', 'world', 'event', 'good', 'day', 'great', 'daily', 'source', 'global', 'current']
Topic 3 = ['audio', 'listen', 'feature', 'option', 'well', 'love', 'good', 'section', 'very', 'want']
Topic 4 = ['news', 'update', 'always', 'present', 'laggy', 'break', 'magazine', 'professional', 'true', 'daily']
Topic 5 = ['news', 'daily', 'great', 'source', 'provide', 'information', 'perspective', 'thank', 'appreciate', 'global']
Topic 6 = ['more', 'very', 'new', 'journalism', 'year', 'good', 'extremely', 'take', 'enjoy', 'recently']
Topic 7 = ['load', 'slow', 'time', 'very', 'open', 'screen', 'issue', 'loading', 'text', 'work']
Topic 8 = ['audio', 'download', 'version', 'new', 'issue', 'content', 'play', 'edition', 'work', 'old']
Topic 9 = ['great', 'news', 'morning', 'easy', 'daily', 'story', 'content', 'more', 'day', 'way']
Topic 10 = ['edition', 'print', 'weekly', 'version', 'magazine', 'well', 'word', 'hope', 'time', 'sense']
Topic 11 = ['content', 'pay', 'think', 'more', 'month', 'really', 'subscription', 'year', 'find', 'news']
Topic 12 = ['subscription', 'ad', 'pay', 'cancel', 'free', 'only', 'content', 'magazine', 'experience', 'year']
Topic 13 = ['day', 'news', 'morning', 'start', 'espresso', 'short', 'great', 'way', 'good', 'world']
Topic 14 = ['page', 'notification', 'scroll', 'open', 'ipad', 'only', 'content', 'screen', 'link', 'great']



Topic 0 = Love it
Topic 1 = Bugs (login)
Topic 2 = Love the Journalism
Topic 3 = Audio?
Topic 4 = Bugs?
Topic 5 = Love the Journalism
Topic 6 = Love the Journalism
Topic 7 = Bugs (loading, slow)
Topic 8 = Audio edition
Topic 9 = Content/espresso?
Topic 10 = Weekly edition
Topic 11 = Subscription/pricing
Topic 12 = Subscription/pricing (angry at ads)
Topic 13 = Espresso
Topic 14 = ?



'''

### Test run 1 with 25 topics

In [ ]:
'''
Topic 3 = ['audio', 'listen', 'version', 'play', 'well', 'content', 'new', 'feature', 'edition', 'download'] - Audio
Topic 17 = ['morning', 'world', 'great', 'news', 'way', 'day', 'perfect', 'love', 'quick', 'minute'] - Love Espresso
Topic 2 = ['news', 'world', 'source', 'good', 'global', 'event', 'daily', 'provide', 'analysis', 'write'] - Journalism
Topic 15 = ['fantastic', 'save', 'daily', 'update', 'solve', 'social', 'problem', 'tech', 'great', 'lay'] - ?
Topic 20 = ['news', 'more', 'great', 'daily', 'time', 'world', 'excellent', 'topic', 'need', 'source'] - ?
Topic 23 = ['download', 'audio', 'play', 'stop', 'work', 'phone', 'very', 'offline', 'bug', 'screen'] - Audio / bug
Topic 16 = ['black', 'function', 'really', 'know', 'good', 'only', 'currency', 'properly', 'more', 'want'] - ?
Topic 6 = ['really', 'enjoy', 'more', 'consume', 'current', 'star', 'format', 'however', 'phone', 'continue'] - ?
Topic 14 = ['page', 'open', 'scroll', 'link', 'search', 'click', 'notification', 'find', 'screen', 'ipad'] - ?
Topic 24 = ['easy', 'news', 'very', 'great', 'daily', 'nice', 'good', 'more', 'morning', 'content'] - 
Topic 1 = ['update', 'content', 'time', 'work', 'issue', 'download', 'load', 'often', 'late', 'fail'] - 
Topic 12 = ['ad', 'pay', 'subscription', 'subscriber', 'experience', 'free', 'magazine', 'service', 'user', 'content'] - too many ads
Topic 11 = ['subscription', 'cancel', 'subscribe', 'pay', 'access', 'digital', 'try', 'content', 'only', 'month'] - subscriptions
Topic 4 = ['write', 'news', 'couple', 'region', 'true', 'understand', 'professional', 'star', 'language', 'concisely'] - 
Topic 5 = ['info', 'present', 'year', 'global', 'touch', 'terrific', 'wise', 'lovely', 'know', 'story']
Topic 9 = ['story', 'more', 'great', 'recommend', 'write', 'definitely', 'insight', 'way', 'busy', 'real']
Topic 13 = ['day', 'news', 'start', 'summary', 'great', 'short', 'morning', 'way', 'espresso', 'good'] - espresso
Topic 19 = ['year', 'weekly', 'magazine', 'world', 'good', 'news', 'issue', 'coverage', 'week', 'daily']
Topic 0 = ['issue', 'year', 'tab', 'favourite', 'great', 'need', 'love', 'very', 'feel', 'work']
Topic 7 = ['load', 'slow', 'time', 'try', 'very', 'content', 'take', 'work', 'new', 'sometimes'] - bugs (slow)



'''


### Test run 1 with 8 topics

In [ ]:
'''
default_configs = {'lda':{'num_topics':8,
                          'passes':300, #2000, # Default 1 just tried 50
                          'iterations':100, # Default 50
                          'random_state':1,
                          'eval_every':10}}


Topic 0 = ['more', 'love', 'year', 'issue', 'great', 'think', 'magazine', 'copy', 'print', 'hard']
Topic 1 = ['subscription', 'pay', 'ad', 'work', 'content', 'time', 'access', 'cancel', 'try', 'subscribe']
Topic 2 = ['news', 'day', 'world', 'great', 'morning', 'good', 'daily', 'way', 'story', 'more']
Topic 3 = ['audio', 'download', 'listen', 'version', 'content', 'play', 'edition', 'new', 'work', 'issue']
Topic 4 = ['news', 'always', 'magazine', 'truly', 'year', 'enjoy', 'write', 'smoothly', 'agree', 'report']
Topic 5 = ['sometimes', 'crn', 'perspective', 'notification', 'usual', 'find', 'user', 'activate', 'page', 'run']
Topic 6 = ['more', 'new', 'content', 'update', 'version', 'good', 'feature', 'phone', 'really', 'need']
Topic 7 = ['load', 'time', 'open', 'screen', 'page', 'content', 'version', 'work', 'only', 'issue']



Topic 0 = Weekly Edition
Topic 1 = Subscription / Payment / Ads
Topic 2 = Espresso / world in brief
Topic 3 = Audio
Topic 4 = ?
Topic 5 = CRN Activation?
Topic 6 = Bugs / Features / Suggestions
Topic 7 = Bugs / Issues



'''